In [3]:
import numpy as np
import pandas as pd
import plotly.express as px
from datetime import date,datetime, timedelta
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels.api as sm
import statsmodels.formula.api as smf
import calendar

In [4]:
df=pd.read_csv('https://raw.githubusercontent.com/andreapiemmi/RepoTransactions_Case/main/repo_trades.csv',sep=';')
df[df.columns[['Date' in c for c in df.columns]]]=df.loc[:,['Date' in c for c in df.columns]].apply(
    lambda x: pd.to_datetime(x.str.replace('.','-',regex=True)),axis=0)
df=df.sort_values(by=['BusinessDate','PurchaseDate']).reset_index(drop=True)
#df

# Question 1

In [5]:
d1=pd.DataFrame()
for i in pd.date_range(date(2020,1,1), periods=366):
    t=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','CashAmount']].groupby(['Term']).sum().squeeze().rename(i)
    d1=pd.concat([d1,pd.DataFrame(t).T],axis=0)
#d1.head()

In [6]:
fig = go.Figure()
for i in d1.columns:
    fig.add_trace(go.Scatter(
        x=d1.index, y=d1[i],
        mode='lines',
        line=dict(width=0.5),
        name=i,
        stackgroup='one',
        text=d1.columns,
        customdata=d1.columns
    ))
fig.add_trace(go.Scatter(x=d1.index,y=d1.sum(axis=1),name='Total',line=dict(color='slategray',dash='dot')))
fig.update_traces(hovertemplate='<b>%{y}</b>')
fig.update_layout(title='Repo Market Outstanding Volume',
                  xaxis_title='Date',
                  yaxis_title='Value in CHF',
                  hovermode="x",
                  template='plotly_white')
fig.show()

In [7]:
summ=d1.copy().apply(lambda x: x/x.sum(), axis=1).mean()
summ

Term
1M    0.128761
1W    0.124469
3M    0.098483
6M    0.053108
ON    0.601755
dtype: float64

# Question 2

In [8]:
selection_Q2=['ON','1W','1M']



d2=pd.DataFrame()
for i in pd.date_range(date(2020,1,1), periods=366):
    t=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','CashAmount','Rate']].groupby(['Term']).apply(
                                  lambda x: np.dot(x['CashAmount']/x['CashAmount'].sum(), x['Rate']).sum() ).squeeze().rename(i)

    #print(t)
    d2=pd.concat([d2,pd.DataFrame(t).T],axis=0)
d2=d2[selection_Q2]

In [9]:
fig=go.Figure()
fig.add_trace(go.Scatter(x=d2.index,y=d2['ON'],name='ON',line=dict(color='teal')))
fig.add_trace(go.Scatter(x=d2.index,y=d2['1W'],name='1W',line=dict(color='darkturquoise')))
fig.add_trace(go.Scatter(x=d2.index,y=d2['1M'],name='1M',line=dict(color='darkseagreen')))
fig.update_traces(hovertemplate='<b>%{y}</b>')
fig.update_layout(title='Volume Weighted Avg. Interest Rate (p.a.) on Repos (ON,1W,1M)',
                  xaxis_title='Date',
                  yaxis_title='Rate <b>p.a. (%)</b>',
                  hovermode="x",
                  template='plotly_white')

# Question 3

In [10]:
d3=pd.DataFrame()
def q3(s):
    return s.quantile(0.75)
def q1(s):
  return s.quantile(0.25)

def standardize(x):
  return (x-x.mean())/x.std()
functions={'Rate':['std', q1, q3]}

for i in pd.date_range(date(2020,1,1), periods=366):
    t=df[df['Term']=='ON'].loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','Rate']].groupby(['Term']).agg(functions).T.reset_index(
                                  level=[0]).drop(columns='level_0').rename(columns={'ON':i}).T
    d3=pd.concat([d3,pd.DataFrame(t)],axis=0)
#d3

In [11]:
fig1 = make_subplots(specs=[[{"secondary_y": True}]])

fig1_p2=go.Figure(go.Scatter(x=d2['ON'].index,y=d2['ON'].values,mode='lines',name='idIR',line_color='slategray',yaxis="y2",opacity=0.4))
fig1_p2.add_traces(go.Scatter(x=d3['q3'].index,y=d3['q3'].values,fill=None,line=dict(width=0),name='75%',
                              yaxis="y2",line_color='slategray',opacity=0.15))
fig1_p2.add_traces(go.Scatter(x=d3['q1'].index,y=d3['q1'].values,fill='tonexty',fillcolor='rgba(112, 128, 144, 0.1)',
                              line=dict(width=0),name='25%',yaxis="y2",line_color='slategray',opacity=0.15))

fig1_p1=px.line(d3['std'], render_mode="webgl",).update_traces(line_color='teal')

fig1.add_traces(fig1_p1.data + fig1_p2.data).update_layout(showlegend=True,
                       template='plotly_white')
fig1.update_traces(hovertemplate='%{y:.2f}')
fig1.layout.yaxis.title="Volatility (Intraday)<b>(%)</b>"
fig1.layout.yaxis.color="teal"
fig1.layout.yaxis2.title="Mean Interest Rate (Intraday) <b>p.a.(%)</b>"
fig1.layout.yaxis2.color = 'darkgray'
fig1.update_layout(hovermode="x unified",template='plotly_white',
                  hoverlabel=dict(bgcolor='rgba(255,255,255,0.1)',font=dict(color='black')))
fig1.show()
#Collect Hoverdata and plot of returns on that day with median + mean to show how the IQR helps understanding skewness of the interest rate distribution


# Question 4

In [12]:
#Select all end & beginning of the month and check for significant mean difference with respect to the rest of the month with non-paired sample test
#Two scenarios envisaged:
#1. For given overnight rates, leverage considerations disincentivize banks to borrow and, hence, the volume of outstanding repos at month-end drops
#2. The SNB anticipates leverage considerations of banks at month end and, hence, lowers rates to prevent excessive fluctuations in volume of
#outstanding transaction
d4=pd.DataFrame()
for i in pd.date_range(date(2020,1,1), periods=366):
  t1=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','CashAmount']].groupby(['Term']).skew().squeeze().rename(i)
  t2=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','Rate']].groupby(['Term']).skew().squeeze().rename(i)
  if 'ON' in t1.index or 'ON' in t2.index:
    temp=pd.concat([t1,t2],axis=1).T['ON'].rename(i).reset_index(drop=True).set_axis(['ON_Volume_skew','ON_Rate_skew'])
    d4=pd.concat([d4,pd.DataFrame(temp).T],axis=0)

d4=d4.join(pd.concat([d1['ON'].rename('ON_Total_Volume'),d2['ON'].rename('ON_Average_Rate')],axis=1),how='left')
for i in d4.index:
  d4.loc[i,'ON_N_repos']=df[df['Term']=='ON'].loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','CashAmount']].shape[0]


Month_Ranges=list(zip( [np.min(d4.index[d4.index.month==m]) for m in np.unique(d4.index.month)],
 [np.max(d4.index[d4.index.month==m]) for m in np.unique(d4.index.month)] ))
d4['Month_End_Dummy']=[*pd.Series([*d4.index]).apply(lambda x: 1 if x in [item[1] for item in Month_Ranges] else 0)]

for lag in range(1,7):
  Range_for_Ones=list( zip(d4['Month_End_Dummy'][d4['Month_End_Dummy']==1].index-timedelta(days=lag),d4.index[d4['Month_End_Dummy']==1]) )
  for i in range(len(Range_for_Ones)):
    selection=d4.index[np.logical_and(d4.index>=Range_for_Ones[i][0],
                                     d4.index<=Range_for_Ones[i][1])]
    d4.loc[selection,'Month_End_Dummy'+str(lag)]=1

#d4[['Dummy' in s for s in d4.columns]]=d4[['Dummy' in s for s in d4.columns]].fillna(0)
d4[d4.columns[['Dummy' in s for s in d4.columns]]]=d4[d4.columns[['Dummy' in s for s in d4.columns]]].fillna(0)

In [13]:
fig=go.Figure(data=[go.Scatter(x=d4[d4['Month_End_Dummy']!=1]['ON_Total_Volume']/d4[d4['Month_End_Dummy']!=1]['ON_N_repos'],y=d4[d4['Month_End_Dummy']!=1]['ON_Average_Rate'],
                                    mode = 'markers',
                                    marker_color=d4[d4['Month_End_Dummy']!=1]['Month_End_Dummy'].replace({1:'indianred',0:'teal'}),
                                    text=[i.date() for i in d4[d4['Month_End_Dummy']!=1].index],hoverinfo = 'text'

                                    )])
fig.add_trace(go.Scatter(x=d4[d4['Month_End_Dummy']==1]['ON_Total_Volume']/d4[d4['Month_End_Dummy']==1]['ON_N_repos'],y=d4[d4['Month_End_Dummy']==1]['ON_Average_Rate'],
                                    mode = 'markers',
                                    marker_color=d4[d4['Month_End_Dummy']==1]['Month_End_Dummy'].replace({1:'indianred',0:'teal'}),
                                    text=[i.date() for i in d4[d4['Month_End_Dummy']==1].index],hoverinfo = 'text'
                                    ))
fig.update_layout(template='plotly_white',xaxis_title='Outstanding Amount',yaxis_title='Rate <b>p.a. (%)</b>')
fig.show()

**Total Volume as Outcome Variable** - Main Result: Only on reporting date Month_End_Dummy has negative coefficient, otherwise always positive. This suggests something, yet one sees **no significant estimated coefficient**

In [14]:
d4['ON_Total_Volume_std']=standardize(d4['ON_Total_Volume'])
d4['ON_Avg_Volume_std']=standardize(d4['ON_Total_Volume_std']/d4['ON_N_repos'])
d4['ON_Average_Rate_std']=standardize(d4['ON_Average_Rate'])
#

In [15]:
summaries={}
summaries['0']=sm.OLS(d4['ON_Total_Volume'],
                      sm.add_constant(d4[['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1').summary()
for i in range(1,7):
  summaries[i]=sm.OLS(d4['ON_Total_Volume'],
                      sm.add_constant(d4[['Month_End_Dummy'+str(i),'ON_Average_Rate']])).fit(cov_type='HC1').summary()

summaries


{'0': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:        ON_Total_Volume   R-squared:                       0.573
 Model:                            OLS   Adj. R-squared:                  0.571
 Method:                 Least Squares   F-statistic:                     463.1
 Date:                Mon, 11 Mar 2024   Prob (F-statistic):          3.83e-100
 Time:                        20:47:31   Log-Likelihood:                -8998.2
 No. Observations:                 362   AIC:                         1.800e+04
 Df Residuals:                     359   BIC:                         1.801e+04
 Df Model:                           2                                         
 Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
 -----------------------------------------------------------

In [16]:
A=sm.OLS(d4['ON_Total_Volume'],
                      sm.add_constant(d4[['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1')
pd.concat([A.params,A.pvalues],axis=1).rename(columns={0:'Coefficient',1:'PValue'})

,Coefficient,PValue
const,-2.881365e+09,1.426569e-01
Month_End_Dummy,-8.764006e+08,8.139475e-01
ON_Average_Rate,4.407408e+10,3.175731e-196


In [17]:
summaries={}
summaries['0']=sm.OLS(d4['ON_Total_Volume']/d4['ON_N_repos'],
                      sm.add_constant(d4[['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1').summary()
for i in range(1,7):
  summaries[i]=sm.OLS(d4['ON_Total_Volume']/d4['ON_N_repos'],
                      sm.add_constant(d4[['Month_End_Dummy'+str(i),'ON_Average_Rate']])).fit(cov_type='HC1').summary()

summaries

{'0': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.438
 Model:                            OLS   Adj. R-squared:                  0.435
 Method:                 Least Squares   F-statistic:                     40.15
 Date:                Mon, 11 Mar 2024   Prob (F-statistic):           1.84e-16
 Time:                        20:47:31   Log-Likelihood:                -6436.7
 No. Observations:                 362   AIC:                         1.288e+04
 Df Residuals:                     359   BIC:                         1.289e+04
 Df Model:                           2                                         
 Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
 -----------------------------------------------------------

In [18]:
d4_copia=d4.copy()#[d4.copy().index.month!=12]
d4_copia['ON_Avg_Outstanding_Volume']=d4['ON_Total_Volume']/d4['ON_N_repos']

quant_formula = 'ON_Total_Volume ~ Month_End_Dummy + ON_Average_Rate'
quant_mod = smf.quantreg(quant_formula, d4_copia)
quantiles = [0.01,0.1, 0.25, 0.5, 0.75, 0.90,0.99]  # Quantiles of interest

quant_results = []
for quantile in quantiles:
    quant_result = quant_mod.fit(q=quantile, max_iter=10000)
    quant_results.append(quant_result)

for quantile, quant_result in zip(quantiles, quant_results):
    print(f"Quantile: {quantile}")
    print(quant_result.summary())


/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.



Quantile: 0.01
                         QuantReg Regression Results                          
Dep. Variable:        ON_Total_Volume   Pseudo R-squared:              0.08264
Model:                       QuantReg   Bandwidth:                   3.298e+10
Method:                 Least Squares   Sparsity:                    5.083e+11
Date:                Mon, 11 Mar 2024   No. Observations:                  362
Time:                        20:47:41   Df Residuals:                      359
                                        Df Model:                            2
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        2.581e+09    4.9e+10      0.053      0.958   -9.38e+10     9.9e+10
Month_End_Dummy  8.126e+08    4.1e+10      0.020      0.984   -7.98e+10    8.14e+10
ON_Average_Rate   5.58e+09   2.86e+10      0.195      0.845   -5.06e+10    6.18e+10
Quantile: 0.

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.



In [19]:
A=pd.DataFrame()
for i in range(len(quantiles)):
  AA=pd.concat([quant_results[i].params, quant_results[i].pvalues],axis=1).rename(columns={0:'Coefficient',
                                                                                          1:'Pvalue'}).loc['Month_End_Dummy',:].rename(quantiles[i])
  A=pd.concat([A,AA],axis=1)
A.T

,Coefficient,Pvalue
0.01,8.126404e+08,0.984195
0.10,-7.459391e+08,0.911975
0.25,-1.937883e-04,1.000000
0.50,-2.370145e+09,0.562656
0.75,-3.848028e+09,0.204602
0.90,-3.883035e+09,0.096702
0.99,1.844567e+10,0.000054


In [20]:
functions={'CashAmount':['median', q1, q3]}
dd=pd.DataFrame()
for i in pd.date_range(date(2020,1,1), periods=366):
    t=df[df['Term']=='ON'].loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['Term','CashAmount']].groupby(['Term']).agg(functions).T.reset_index(
                                  level=[0]).drop(columns='level_0').rename(columns={'ON':i}).T
    dd=pd.concat([dd,t],axis=0)
dd=pd.concat([d4,dd],axis=1)

fig1=go.Figure()
for month in np.arange(1,13):
  if month!=12:
    switch='legendonly'
  else:
    switch=True
  t=dd[dd.index.month==month].reset_index().set_index(np.arange(1,len(dd[dd.index.month==month])+1)/dd[dd.index.month==month].shape[0])
  fig1.add_trace(go.Scatter(x=t['ON_Total_Volume'].index,y=t['ON_Total_Volume']/t['ON_N_repos'],name=str(calendar.month_abbr[month]),
                           visible=switch))
  fig1.add_traces(go.Scatter(x=t['q3'].index,y=t['q3'].values,fill=None,line=dict(width=0),name=str(calendar.month_abbr[month])+'75%',
                              line_color='slategray',opacity=0.15,visible=switch))
  fig1.add_traces(go.Scatter(x=t['q1'].index,y=t['q1'].values,fill='tonexty',fillcolor='rgba(112, 128, 144, 0.1)',
                              line=dict(width=0),name=str(calendar.month_abbr[month])+'25%',line_color='slategray',opacity=0.15,visible=switch))
  fig1.add_traces(go.Scatter(x=t['median'].index,y=t['median'].values,name=str(calendar.month_abbr[month])+'50%',visible=switch))
  fig1.update_layout(template='plotly_white')
fig1

In [23]:
figu=make_subplots(rows=1,cols=2)
figu.add_traces(fig.data,1,1)
figu.add_traces(fig1.data,1,2)
figu.update_layout(template='plotly_white')
figu.update_xaxes(title_text="Oustanding Volune", row=1, col=1)
figu.update_yaxes(title_text="Interest Rate p.a.(%)", row=1, col=1)
figu.update_yaxes(title_text="Outstanding Volume", row=1, col=2)

In [25]:
d4_DecOut=d4[d4.index.month!=12]
summaries={}
summaries['0']=sm.OLS(d4_DecOut['ON_Total_Volume'],
                      sm.add_constant(d4_DecOut[['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1').summary()
for i in range(1,7):
  summaries[i]=sm.OLS(d4_DecOut['ON_Total_Volume'],
                      sm.add_constant(d4_DecOut[['Month_End_Dummy'+str(i),'ON_Average_Rate']])).fit(cov_type='HC1').summary()

summaries


{'0': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:        ON_Total_Volume   R-squared:                       0.025
 Model:                            OLS   Adj. R-squared:                  0.019
 Method:                 Least Squares   F-statistic:                     1.565
 Date:                Mon, 11 Mar 2024   Prob (F-statistic):              0.211
 Time:                        20:51:05   Log-Likelihood:                -8257.7
 No. Observations:                 332   AIC:                         1.652e+04
 Df Residuals:                     329   BIC:                         1.653e+04
 Df Model:                           2                                         
 Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
 -----------------------------------------------------------

**No significance gain when looking at linear regressions with outstanding amount as outcome variable**

In [26]:
summaries={}
summaries['0']=sm.OLS(d4_DecOut['ON_Total_Volume']/d4_DecOut['ON_N_repos'],
                      sm.add_constant(d4_DecOut[['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1').summary()
for i in range(1,7):
  summaries[i]=sm.OLS(d4_DecOut['ON_Total_Volume']/d4_DecOut['ON_N_repos'],
                      sm.add_constant(d4_DecOut[['Month_End_Dummy'+str(i),'ON_Average_Rate']])).fit(cov_type='HC1').summary()

summaries

{'0': <class 'statsmodels.iolib.summary.Summary'>
 """
                             OLS Regression Results                            
 Dep. Variable:                      y   R-squared:                       0.011
 Model:                            OLS   Adj. R-squared:                  0.005
 Method:                 Least Squares   F-statistic:                     1.721
 Date:                Mon, 11 Mar 2024   Prob (F-statistic):              0.180
 Time:                        20:51:05   Log-Likelihood:                -5751.5
 No. Observations:                 332   AIC:                         1.151e+04
 Df Residuals:                     329   BIC:                         1.152e+04
 Df Model:                           2                                         
 Covariance Type:                  HC1                                         
                       coef    std err          z      P>|z|      [0.025      0.975]
 -----------------------------------------------------------

**Regressions with Average Repo Outstanding Amount lose significance throughout all consideration windows**

In [27]:
#Check Again
sm.OLS(d4[d4.index.month!=12]['ON_Total_Volume']/d4[d4.index.month!=12]['ON_N_repos'],
       sm.add_constant(d4[d4.index.month!=12][['Month_End_Dummy','ON_Average_Rate']])).fit(cov_type='HC1').summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.721
Date:                Mon, 11 Mar 2024   Prob (F-statistic):              0.180
Time:                        20:51:05   Log-Likelihood:                -5751.5
No. Observations:                 332   AIC:                         1.151e+04
Df Residuals:                     329   BIC:                         1.152e+04
Df Model:                           2                                         
Covariance Type:                  HC1                                         
===================================================================================
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
const             1.78e+08   8.55e+06     20.831      0.000    1.61e+08    1.95e+08
Month_End_Dummy  -1.49e+06    2.1e+06     -0.708      0.479   -5.61e+06    2.63e+06
ON_Average_Rate -8.179e+06   4.96e+06     -1.649      0.099   -1.79e+07    1.54e+06
==============================================================================
Omnibus:                        5.584   Durbin-Watson:                   0.088
Prob(Omnibus):                  0.061   Jarque-Bera (JB):                5.348
Skew:                           0.300   Prob(JB):                       0.0690
Kurtosis:                       3.160   Cond. No.                         40.8
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [28]:
d4_copia=d4_DecOut.copy()#[d4.copy().index.month!=12]
d4_copia['ON_Avg_Outstanding_Volume']=d4['ON_Total_Volume']/d4['ON_N_repos']

quant_formula = 'ON_Avg_Outstanding_Volume ~ Month_End_Dummy + ON_Average_Rate'
quant_mod = smf.quantreg(quant_formula, d4_copia)
quantiles = [0.05,0.1, 0.25, 0.5, 0.75, 0.90,0.95]  # Quantiles of interest

quant_results = []
for quantile in quantiles:
    quant_result = quant_mod.fit(q=quantile, max_iter=10000)
    quant_results.append(quant_result)

for quantile, quant_result in zip(quantiles, quant_results):
    print(f"Quantile: {quantile}")
    print(quant_result.summary())


/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.



Quantile: 0.05
                             QuantReg Regression Results                             
Dep. Variable:     ON_Avg_Outstanding_Volume   Pseudo R-squared:               0.1257
Model:                              QuantReg   Bandwidth:                   5.023e+06
Method:                        Least Squares   Sparsity:                    6.922e+07
Date:                       Mon, 11 Mar 2024   No. Observations:                  332
Time:                               20:51:10   Df Residuals:                      329
                                               Df Model:                            2
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        2.289e+08   1.79e+07     12.799      0.000    1.94e+08    2.64e+08
Month_End_Dummy -3.495e+06   6.13e+06     -0.571      0.569   -1.55e+07    8.55e+06
ON_Average_Rate -4.434e+07   1.01e+07     -4.37

/usr/local/lib/python3.10/dist-packages/statsmodels/regression/quantile_regression.py:191: IterationLimitWarning:

Maximum number of iterations (10000) reached.



**Further significance gain by quantile regression focusing on upper quantiles, like 75%, 90% and 95%**

# Question 5

In [29]:


d5=pd.DataFrame()
for i in pd.date_range(date(2020,1,1), periods=366):
    t1=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['CashTakerID','Term','CashAmount']].groupby(['CashTakerID','Term']).sum()

    t2=df.loc[np.logical_and(df['PurchaseDate']<=i,
                              i<df['RepurchaseDate'])][['CashTakerID','Term','CashAmount','Rate']].groupby(['CashTakerID','Term']).apply(
                                  lambda x: np.dot(x['CashAmount']/x['CashAmount'].sum(), x['Rate']).sum() ).rename('Rate')

    t=pd.concat([t1,t2],axis=1)
    t['Date']=i
    t=t.loc[[item[1]=='ON' for item in t.index],:].reset_index()
    d5=pd.concat([d5,t],axis=0)
d5=d5.drop(columns=['Term']).reset_index(drop=True)
#d5=d5.set_index(['CashTakerID','Date'])
d5=pd.merge(d5,
         d5[['Date','CashAmount','Rate']].groupby(['Date']).mean().rename(columns={'CashAmount':'CashAmount_Mean-per-Date',
                                                                                   'Rate':'Rate_Mean-per-Date'}) ,
         how='left', left_index=False, right_index=True, left_on='Date')

In [30]:
d5

,CashTakerID,CashAmount,Rate,Date,CashAmount_Mean-per-Date,Rate_Mean-per-Date
0,1,1.393000e+09,2.079454,2020-01-04,8.198000e+08,2.097457
1,2,1.330000e+09,2.002707,2020-01-04,8.198000e+08,2.097457
2,3,1.700000e+08,1.982941,2020-01-04,8.198000e+08,2.097457
3,4,4.650000e+08,2.060108,2020-01-04,8.198000e+08,2.097457
4,5,8.500000e+08,2.258851,2020-01-04,8.198000e+08,2.097457
...,...,...,...,...,...,...
3581,6,1.418377e+08,-0.640865,2020-12-31,2.183203e+08,-0.273422
3582,7,9.097819e+07,-0.024987,2020-12-31,2.183203e+08,-0.273422
3583,8,2.269190e+08,-0.087570,2020-12-31,2.183203e+08,-0.273422
3584,9,3.331143e+08,-0.174851,2020-12-31,2.183203e+08,-0.273422


In [31]:

prova=d5.copy()
prova['CashAmount_Demeaned_Std']=standardize(prova['CashAmount']-prova['CashAmount_Mean-per-Date'])
prova['Rate_Demeaned']=prova['Rate']-prova['Rate_Mean-per-Date']
CashTaker_dummies = pd.get_dummies(prova['CashTakerID'])
X, Y= prova[['CashAmount_Demeaned_Std']].join(CashTaker_dummies.drop(columns=[10])), prova[['Rate_Demeaned']]

In [32]:
model=sm.OLS(Y, sm.add_constant(X)).fit(cov_type='HC1')

In [33]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          Rate_Demeaned   R-squared:                       0.347
Model:                            OLS   Adj. R-squared:                  0.346
Method:                 Least Squares   F-statistic:                     181.6
Date:                Mon, 11 Mar 2024   Prob (F-statistic):          6.37e-310
Time:                        20:51:22   Log-Likelihood:                 2551.2
No. Observations:                3586   AIC:                            -5080.
Df Residuals:                    3575   BIC:                            -5012.
Df Model:                          10                                         
Covariance Type:                  HC1                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                      -0.0702      0.009     -7.509      0.000      -0.089      -0.052
CashAmount_Demeaned_Std     0.0121      0.002      6.247      0.000       0.008       0.016
1                           0.0599      0.011      5.561      0.000       0.039       0.081
2                          -0.0065      0.011     -0.597      0.551      -0.028       0.015
3                           0.0648      0.011      6.011      0.000       0.044       0.086
4                           0.0888      0.011      8.280      0.000       0.068       0.110
5                           0.1622      0.012     13.785      0.000       0.139       0.185
6                          -0.0130      0.011     -1.156      0.248      -0.035       0.009
7                           0.0412      0.010      3.937      0.000       0.021       0.062
8                           0.2834      0.011     24.743      0.000       0.261       0.306
9                           0.0225      0.011      2.023      0.043       0.001       0.044
==============================================================================
Omnibus:                      218.001   Durbin-Watson:                   2.179
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              315.038
Skew:                          -0.525   Prob(JB):                     3.89e-69
Kurtosis:                       4.003   Cond. No.                         10.9
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [34]:
Dummy_Coeff_CI=pd.concat( [model.params.iloc[2:].rename('CashTaker_Dummy_Coeff'),
            model.conf_int(alpha=0.02).rename(columns={0:'1%',1:'99%'}).iloc[2:,:]] ,axis=1)
Dummy_Coeff_CI

,CashTaker_Dummy_Coeff,1%,99%
1,0.059896,0.034839,0.084953
2,-0.006521,-0.031935,0.018893
3,0.064819,0.039733,0.089905
4,0.088775,0.063834,0.113717
5,0.162213,0.134838,0.189588
6,-0.012972,-0.039075,0.013131
7,0.041198,0.016853,0.065542
8,0.283414,0.256768,0.310061
9,0.022462,-0.003371,0.048295


In [37]:
diff = [h - l for h, l in zip(Dummy_Coeff_CI['99%'], Dummy_Coeff_CI['1%'])]
fig = go.Figure()
fig.add_trace(go.Scatter(x=Dummy_Coeff_CI.index, y=Dummy_Coeff_CI['99%'], mode='markers',
                         name='UB(99%)', marker_color='#c92a52', marker_symbol='triangle-down', marker_size=15))
fig.add_trace(go.Scatter(x=Dummy_Coeff_CI.index, y=Dummy_Coeff_CI['CashTaker_Dummy_Coeff'],
                         mode='markers', name='Coefficient', marker_color='black', marker_line_width=1))
fig.add_trace(go.Scatter(x=Dummy_Coeff_CI.index, y=Dummy_Coeff_CI['1%'], mode='markers',
                         name='LB (1%)', marker_color='#4d70c9', marker_symbol='triangle-up', marker_size=15))

fig.add_trace(go.Bar(x=Dummy_Coeff_CI.index, y=diff, base=Dummy_Coeff_CI['1%'], width=0.01, marker_color='black', showlegend=False))
fig.update_layout(template='plotly_white',xaxis_title='Borrower ID',yaxis_title="Competitor's Differential Rate")
fig.show()